In [ ]:
import pandas as pd
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
import chromedriver_autoinstaller
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])
# opt.add_argument("--headless")
# chromedriver_autoinstaller.install()

In [ ]:
# !pip install selenium
# !pip install chromedriver
# !pip install chromedriver_autoinstaller

In [ ]:

scrape_links = {
    "INSIDE LACROSSE (GIRLS)": "https://www.insidelacrosse.com/recruiting/allcommitments/girls",               
    "INSIDE LACROSSE (BOYS)": "https://www.insidelacrosse.com/recruiting/allcommitments"
    }


In [ ]:
output_folder_name="../data/insidelacrosse"
if (not os.path.exists(output_folder_name)):
    os.mkdir(output_folder_name)

In [ ]:
driver = webdriver.Chrome(options=opt)

In [ ]:
def load_dynamic_main_table(
    web_driver,
    url,
    range_input=1000,
    sleep_interval=1
    ):
    web_driver = web_driver
    url = url
    web_driver.get(url)
    sleep(5)
    print("scrolling to laod complete main table")
    previous_length = 0
    counter =0
    for x in range (range_input):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(sleep_interval)
        results=driver.find_elements(by=By.XPATH, value='.//table[@class="table box table-striped"]')
        if previous_length <= len(results):
            counter += 1
        else:
            counter = 0
        previous_length = len(results)
        table_html = results[-1].get_attribute("outerHTML")
        # check if table is empty
        df = pd.read_html(table_html)[0]
        if df.shape[0] == 0:
            break
        if counter > 20:
            print("table loading finished")
            break
    return True

In [ ]:
def fetch_main_table_from_loaded_page():
    final_dfs=[]
    print("retrieving all data.... \n")

    #     get all table's data and concatenate them
    results=driver.find_elements(by=By.XPATH, value='.//table[@class="table box table-striped"]')
    for each_table in results:
        # Use Pandas to read HTML and convert it to a DataFrame
        df = pd.read_html(each_table.get_attribute("outerHTML"))[0]
        links = []
        for row_index, row in df.iterrows():
            link_elements = each_table.find_elements(by=By.XPATH, value=f'.//tr[{row_index + 1}]/td[1]//a')
            # Extract the href attribute from the first link element, if any
            link = link_elements[0].get_attribute("href") if link_elements else None
            links.append(link)
        # Add a new column to the DataFrame with the extracted links
        df["Link"] = links
        final_dfs.append(df)
    result_df=pd.concat(final_dfs, axis=0, ignore_index=True)
    return result_df

In [34]:

def scrape_player_url_data(url):
    driver = webdriver.Chrome(options=opt)  
    driver.get(url+"?active=teams")
    result_dict={}
    try:
        # Wait for the presence of the div 
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/table[@class="table table-striped box m-b-0"]'))
        )
        sleep(1)
        results=driver.find_elements(by=By.XPATH, value='.//table[@class="table table-striped box m-b-0"]')
        for table in results:
            header_elements = table.find_elements(by=By.XPATH, value=".//thead//th")  # Adjust the XPath as needed

            # Extract the text content from each header element
            column_names = [header.text for header in header_elements]
            if "Club" in column_names:
                df = pd.read_html(table.get_attribute("outerHTML"))[0]
                result_dict = df.iloc[:, :2].to_dict(orient='list')
                result_dict={f"Season_{season}": club for season, club in zip(result_dict['Season'], result_dict['Club'])}
    except:
        pass
    driver.quit()
    return result_dict


In [ ]:
if load_dynamic_main_table(driver,scrape_links['INSIDE LACROSSE (GIRLS)'],sleep_interval=3):
    scrapped_df=fetch_main_table_from_loaded_page()

In [33]:
scrape_player_url_data(scrapped_df['Link'][0])

{'Season_2024': 'Yellow Jackets 2025 Lomo',
 'Season_2023': 'Yellow Jackets 2025 Lomo',
 'Season_2022': 'Yellow Jackets 2023 Rella'}

In [ ]:
driver.close()

### load complete table page 